## Nonlinear DEGs from PerturbSeq

#### Background
- We have got a reactome similarity matrix between the perturbed genes, both in desiase and healthly.
- We know that DEGs-based similarity matrix has a statistically significant overlap with the reactome similarty matrix. 
- We want to extract knowledge from the perturbSeq dataset which would overlap with the reactome pathways.
- We already tried to build a VAE-based model learning an interpretable representation of every perturbation. Even though it gives promising results (statistically significant overlap with reactome similarity mtx), it requires further tuning/refinement as the stability of the solution is low. Specifically, adjustment of the loss function and the learning concept might be needed.
- At this point we operate on single-gene perturbations only due to their broader availability. We might use 2-gene perturbations for validation of our hypotheses and expansion of the method in the following iterations.

#### The Why
- Why do we believe it is reasonable to do? DEGs are computed using t-test and have a statistically significant overlap with reactome. If we incorporate nonlinearity into DEGs in some way, we might get an even stronger result.
- Why do we want to do it? It would give us a subset of perspective relationships between genes whose exploration would potentially lead to new biological findings.
- **Optional addon:** We know that statistical methods used for DEGs do not use feature interactions (they might but it is uncommon). We could introduce a model which would focus on feature interactions only. This way we could have effect from the single-feature effects and interactions separately. 

#### The How
- What is the simplest way? Take a binary boosting trees classifier (lgbm or catboost), and find the top important features for every perturbation, for the model classifying into ctrl vs perturbed state. Ofc, one can play with SHAPs, etc. 
- What is a more advanced approach? Having a similarity matrix from the previous step, we can refine it by building binary classfiers for perturbation A vs perturbation B task. This would allow to refine differences between the perturbations. The classifier performance and its feature importances (or SHAPs) would be used for the refinement.
- Check the result consistency with HPT and swapping the classifier.

#### Performance of the approach
- How to assess the success of the approach? Hmm...
  - Check the overlap with the DEGs - for exploration purposes only, does not allow to assess usability
  - Consider TF pairs which were not explored together and are similar according to our approach. Play with them in the wetlab to get a proof that they are connected? Is it of any use at all?
  
#### Thoughts and feedback
- Network analysis: Construct gene interaction networks based on your results and compare their properties (e.g., modularity, centrality) with known biological networks.
- Consider using multi-class classification instead of multiple binary classifiers to potentially capture global patterns.
- Implement a hierarchical classification scheme to group similar perturbations - this is basically a sequence of splits by multiple models following the known hierarchy. We have got the hierarchy from the VAE-based model or we can get one from the hierarchical clustering. Now, we can refine the paths. Here are several approaches we can consider for that:
    - Start with your initial hierarchy.
    - Train classifiers at each node and evaluate their performance.
    - Identify nodes where performance is poor.
    - For these problematic nodes, consider:
        - a) Splitting the node further if it's too heterogeneous.
        - b) Merging with a sibling node if they're too similar.
        - c) Adjusting the perturbations within the node - check the ones contributing to the error the most and change their class.
    - Repeat this process iteratively.
</br>   </br>   
- **Reasons for only around 10% overlap between DEGs and the pathways?**
  - a) The genes are regulated by the same TF/protein/signalling mechanism but have nothing else in common (called indirect relationships).
    - This can still be reported in pathways DBs, e.g. WikiPathways, right? 
  - b) Incomplete pathway annotations - they might be actually related but we do not know that yet.
    - This is our research gap
  - c) Spurious correlations, cell-type or condition-specific, technical artifacts, genomic proximity, temporal nature of the coexpression.
    - We account for it by analysing multiple datasets with different cell types, timelines, conditions, etc. This should address most of these issues.  
    
#### To decide on the path, we need to see what brings us to the unknown biology faster!

### Imports

In [1]:
import scanpy as scp
import pandas as pd
import numpy as np
import catboost as cb
from tqdm import tqdm
from scipy import sparse
from sklearn.model_selection import train_test_split

from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from collections import Counter
import lightgbm as lgb
import matplotlib.pyplot as plt
import umap
from sklearn.model_selection import StratifiedKFold

### Magics

In [3]:
import scanpy as scp
import pandas as pd
import numpy as np
import catboost as cb
from tqdm import tqdm
from scipy import sparse
from sklearn.model_selection import train_test_split

from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from collections import Counter
import lightgbm as lgb
import matplotlib.pyplot as plt
import umap
from sklearn.model_selection import StratifiedKFold

import ot
from sklearn.decomposition import PCA
from IPython.display import clear_output
from sklearn.preprocessing import QuantileTransformer

### Magics

In [4]:
GENE_PER_CELL_BINNING = False
N_BINS = 1000
N_ITER = 50
TOP_N_GENES = 576

### Prepare data

In [5]:
adata = scp.read_h5ad('./data/Norman_2019/norman_umi_go/perturb_processed.h5ad')

In [6]:
TFs = pd.read_csv('little_data/TF_db.csv',index_col=0)

In [7]:
adata = adata[:, adata.var.loc[adata.var.index.isin(TFs['Ensembl ID'].values)].index]

In [8]:
## Following the scGPT paper, we bin the genes within cell. 

def bin_nonzero_values(arr, num_bins):
    # Filter out non-zero values
    nonzero_vals = arr[arr != 0]
    
    # Calculate bin edges
    bin_edges = np.linspace(nonzero_vals.min(), nonzero_vals.max(), num_bins)
    
    # Bin the values
    binned_values = np.zeros_like(arr)
    binned_nonzero = np.digitize(nonzero_vals, bin_edges)
    binned_values[arr != 0] = binned_nonzero
    
    return binned_values

# Example usage
arr = np.random.randint(low=0, high=100, size=100)
num_bins = 3
binned_values = bin_nonzero_values(arr, num_bins)
print(set(binned_values))

{0, 1, 2, 3}


In [9]:
scp.pp.normalize_total(adata, exclude_highly_expressed=True)
scp.pp.log1p(adata)
scp.pp.highly_variable_genes(adata, n_top_genes=TOP_N_GENES,subset=False)

/hpcfs/users/a1234104/miniconda3/envs/py310torch/lib/python3.10/site-packages/scanpy/preprocessing/_normalization.py:206: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [10]:
if GENE_PER_CELL_BINNING:
    tempy = adata.X.toarray()
    
    for c in tqdm(range(adata.X.shape[0])):
        tempy[c,:] = bin_nonzero_values(tempy[c,:], N_BINS)
    
    adata.X = sparse.csr_matrix(tempy)
    del tempy

In [11]:
y = adata.obs.condition.values.astype(str)
X = adata.X.toarray()

In [12]:
gene_num_map = ['ctrl']

for rec in tqdm(y):
    comps = rec.split('+')
    for c in comps:
        if c not in gene_num_map:
            gene_num_map.append(c)

100%|██████████| 91205/91205 [00:00<00:00, 894821.35it/s]


In [ ]:
# #################### Feature Importance Approach #####################
# from sklearn.model_selection import train_test_split
# from catboost import CatBoostClassifier

# # Assuming you have X (features) and y (labels) already defined
# # Replace with your actual data

# # Create a dictionary to store top N important features for each dataset
# top_features_dict = {}
# feats = adata.var.gene_name.values
# # Define the number of top features to select
# N = 20

# # Iterate over each gene in gene_num_map
# for g in tqdm(gene_num_map[1:]):
#     # Create a mask for the current gene
#     mask = np.array([(g in element and 'ctrl' in element) or element == 'ctrl' for element in y]).astype(bool)
    
#     # Filter the data for the current gene
#     X_temp = X[mask]
#     y_temp = y[mask]
#     y_temp[y_temp != 'ctrl'] = 1
#     y_temp[y_temp == 'ctrl'] = 0
    
#     # Split the data into train, validation, and test sets
#     X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.15, random_state=42)
#     #X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=42)
    
#     # Initialize the CatBoost classifier
#     #clf = CatBoostClassifier(loss_function='Logloss') #iterations=300, depth=7, learning_rate=0.1, 
#     clf = CatBoostClassifier(
#                                 iterations=1000,
#                                 learning_rate=0.05,
#                                 depth=6,
#                                 l2_leaf_reg=3,
#                                 random_strength=1,
#                                 bagging_temperature=1,
#                                 od_type='Iter',
#                                 od_wait=50,
#                                 verbose=0,
#                                 auto_class_weights='Balanced'
#                             )
    
#     # Fit the classifier on the training data
#     clf.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=10, verbose=0)
    
#     # Get feature importances
#     feature_importances = clf.feature_importances_
    
#     # Get indices of top N important features
#     top_feature_indices = np.argsort(feature_importances)[-N:]
    
#     # Get the actual feature names
#     top_features = feats[top_feature_indices]
    
#     # Store the top features in the dictionary
#     top_features_dict[g] = top_features

# # Create a pandas DataFrame from the dictionary
# top_features_df = pd.DataFrame.from_dict(top_features_dict).T

# # import shap

# # # Assuming you have X (features) and y (labels) already defined
# # # Create a dictionary to store top N important features for each dataset
# # top_features_dict = {}
# # feats = adata.var.gene_name.values

# # # Define the number of top features to select
# # N = 50

# # # Iterate over each gene in gene_num_map
# # for g in tqdm(gene_num_map[1:]):
# #     # Create a mask for the current gene
# #     mask = np.array([(g in element and 'ctrl' in element) or element == 'ctrl' for element in y]).astype(bool)
    
# #     # Filter the data for the current gene
# #     X_temp = X[mask]
# #     y_temp = y[mask]
# #     y_temp = np.where(y_temp == 'ctrl', 0, 1)  # 'ctrl' as 0, 'perturbation' as 1
    
# #     # Split the data into train and test sets
# #     X_train, X_test, y_train, y_test = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)
    
# #     # Initialize the CatBoost classifier
# #     #clf = CatBoostClassifier(loss_function='Logloss', verbose=0,  auto_class_weights='Balanced')
# #     clf = CatBoostClassifier(
# #                                 iterations=2000,
# #                                 learning_rate=0.02,
# #                                 depth=10,
# #                                 l2_leaf_reg=1,
# #                                 random_strength=0.5,
# #                                 rsm=0.8,
# #                                 one_hot_max_size=10,
# #                                 leaf_estimation_iterations=10,
# #                                 max_ctr_complexity=3,
# #                                 verbose=0,
# #                                 auto_class_weights='Balanced'
# #                             )
    
# #     # Fit the classifier on the training data
# #     clf.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=10, verbose=0)
    
# #     # Make predictions on the test set
# #     y_pred = clf.predict(X_test)
    
# #     # Get indices of correctly classified samples
# #     correct_indices = np.where(y_pred == y_test)[0]
    
# #     # Filter X_test for correctly classified samples
# #     X_test_correct = X_test[correct_indices]
# #     y_test_correct = y_test[correct_indices]
    
# #     # Get control samples to use as reference
# #     X_ctrl = X_test_correct[y_test_correct == 0]
    
# #     # Initialize the SHAP explainer with control samples as reference
# #     explainer = shap.TreeExplainer(clf, data=X_ctrl, feature_perturbation = 'interventional')
    
# #     # Calculate SHAP values for correctly classified perturbation samples
    
# #     X_pert = X_test_correct[y_test_correct == 1]
# #     y_pert = y_test_correct[y_test_correct == 1]
    
# #     if X_pert.shape[0]<1:
# #         print(f'{g} has no correctly classified perturbed records')
# #         continue
    
# #     shap_values = explainer.shap_values(X_pert)
    
# #     # SHAP values are returned as a list of arrays, one for each class
# #     # We're interested in the SHAP values for the 'perturbation' class (index 1)
# #     shap_values_perturbation = shap_values[y_pert==1]
    
# #     # Calculate mean absolute SHAP values across samples
# #     mean_shap_values = np.median(np.abs(shap_values_perturbation), axis=0)
    
# #     # Calculate the maximum SHAP value
# #     max_shap_value = np.max(mean_shap_values)
    
# #     # Get indices of features with impact > 10% of max impact
# #     significant_feature_indices = np.where(mean_shap_values > 0.35 * max_shap_value)[0]
    
# #     # Sort these indices by SHAP value (descending order)
# #     sorted_significant_indices = significant_feature_indices[np.argsort(-mean_shap_values[significant_feature_indices])]
    
# #     # Get the actual feature names (up to N features)
# #     top_features = feats[sorted_significant_indices] #[:N]
    
# #     # Store the top features in the dictionary
# #     top_features_dict[g] = top_features

# # # Create a pandas DataFrame from the dictionary
# # #top_features_df = pd.DataFrame.from_dict(top_features_dict).T

# sim_mtx = pd.DataFrame(index = top_features_df.index, columns = top_features_df.index)
# for g1 in tqdm(top_features_df.index):
#     for g2 in top_features_df.index:
        
#         t1 = top_features_dict[g1].tolist()
#         t2 = top_features_dict[g2].tolist()

#         # m = top_features_df.loc[g1].isin(t2)
#         # d1 = m.loc[m==True].index.values

#         # m = top_features_df.loc[g2].isin(t1)
#         # d2 = m.loc[m==True].index.values
        
#         sim_mtx.loc[g1,g2] = float(len(set(t1).intersection(set(t2))))


# sim_mtx.to_csv('./little_data/CatBoost_sim_mtx.csv')

In [105]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from collections import defaultdict

# Assuming you have X (features), y (labels), and adata already defined

def get_top_features(X, y, feats, N=20):
    top_features_dict = {}
    
    clf = CatBoostClassifier(
        iterations=1000,
        learning_rate=0.05,
        depth=6,
        l2_leaf_reg=3,
        random_strength=1,
        bagging_temperature=1,
        od_type='Iter',
        od_wait=50,
        verbose=0,
        auto_class_weights='Balanced'
    )

    for g in tqdm(gene_num_map[1:]):
        mask = np.array([(g in element and 'ctrl' in element) or element == 'ctrl' for element in y])
        X_temp, y_temp = X[mask], y[mask]
        y_temp = np.where(y_temp == 'ctrl', 0, 1)
        
        X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.15, random_state=42)
        
        clf.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=10, verbose=0)
        
        top_feature_indices = np.argsort(clf.feature_importances_)[-N:]
        top_features_dict[g] = feats[top_feature_indices]
    
    return pd.DataFrame.from_dict(top_features_dict).T

def compute_similarity_matrix(top_features_df):
    sim_mtx = pd.DataFrame(index=top_features_df.index, columns=top_features_df.index)
    for g1 in tqdm(top_features_df.index):
        for g2 in top_features_df.index:
            sim_mtx.loc[g1, g2] = len(set(top_features_df.loc[g1]) & set(top_features_df.loc[g2]))
    return sim_mtx

def build_binary_classifiers(X, y, sim_mtx, quantile=0.95):
    performance_mtx = pd.DataFrame(index=sim_mtx.index, columns=sim_mtx.index)
    clf = CatBoostClassifier(iterations=500, learning_rate=0.05, depth=6, verbose=0)
    threshold = np.quantile(sim_mtx.values.ravel(), quantile)
    
    for g1 in tqdm(sim_mtx.index):  
        # Find similar genes based on the quantile threshold
        similar_genes = sim_mtx.index[sim_mtx[g1] >= threshold]
        
        for g2 in similar_genes:
            if g1 != g2:
                mask = np.array([g1 in str(element) or g2 in str(element) for element in y])
                X_temp, y_temp = X[mask], y[mask]
                y_temp = np.array([1 if g1 in str(label) else 0 for label in y_temp])

                # Only proceed if we have enough samples of each class
                if np.min(np.bincount(y_temp)) >= 20:  # Ensure at least 10 samples in each class
                    X_train, X_test, y_train, y_test = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42, stratify=y_temp)
                    
                    clf.fit(X_train, y_train, verbose=False)
                    y_pred = clf.predict_proba(X_test)[:, 1]
                    
                    try:
                        performance_mtx.loc[g1, g2] = roc_auc_score(y_test, y_pred)
                    except ValueError:
                        # This can happen if y_test contains only one class
                        performance_mtx.loc[g1, g2] = np.nan

    return performance_mtx

In [ ]:
# Main execution
feats = adata.var.gene_name.values
top_features_df = get_top_features(X, y, feats)
sim_mtx = compute_similarity_matrix(top_features_df)
sim_mtx.to_csv('./little_data/CatBoost_sim_mtx.csv')

In [109]:
sim_mtx = pd.read_csv('./little_data/CatBoost_sim_mtx.csv',index_col=0)

In [110]:
performance_mtx = build_binary_classifiers(X, y, sim_mtx, quantile = 0.95)
performance_mtx.to_csv('./little_data/CatBoost_performance_mtx.csv')

100%|██████████| 105/105 [1:07:26<00:00, 38.54s/it]


In [112]:
performance_mtx.fillna(0)

/mnt/tmp_local/lls_2376517/ipykernel_3822717/1942265003.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  performance_mtx.fillna(0)


,TSC22D1,KLF1,MAP2K6,CEBPE,RUNX1T1,MAML2,CBL,PTPN9,TGFBR2,ETS2,...,ATL1,NIT1,CDKN1B,PTPN13,HOXA13,CITED1,PRDM1,HK2,CDKN1C,EGR1
TSC22D1,0.0,0.000000,0.000000,0.0,0.0,0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0,0.0,0.0
KLF1,0.0,0.000000,0.000000,0.0,0.0,0,0.966037,0.966848,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.969909,0,0.0,0.0
MAP2K6,0.0,0.000000,0.000000,0.0,0.0,0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.893898,0.953321,0,0.0,0.0
CEBPE,0.0,0.000000,0.000000,0.0,0.0,0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0,0.0,0.0
RUNX1T1,0.0,0.000000,0.000000,0.0,0.0,0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CITED1,0.0,0.000000,0.897639,0.0,0.0,0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0,0.0,0.0
PRDM1,0.0,0.983938,0.952210,0.0,0.0,0,0.957248,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0,0.0,0.0
HK2,0.0,0.000000,0.000000,0.0,0.0,0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0,0.0,0.0
CDKN1C,0.0,0.000000,0.000000,0.0,0.0,0,0.000000,0.000000,0.0,0.0,...,0.883228,0.0,0.0,0.0,0.0,0.000000,0.000000,0,0.0,0.0
